# LOGISTIC REGRESSION -- prediciting which solver method(out of newton-cg,lbfgs,liblinear) is good for our data, and also predicting the jaccardian score and log loss 

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [124]:
df=pd.read_csv(r"C:\Users\91755\Downloads\ChurnData.csv")

In [125]:
df.head(7)

,tenure,age,address,income,ed,employ,equip,callcard,wireless,longmon,...,pager,internet,callwait,confer,ebill,loglong,logtoll,lninc,custcat,churn
0,11.0,33.0,7.0,136.0,5.0,5.0,0.0,1.0,1.0,4.40,...,1.0,0.0,1.0,1.0,0.0,1.482,3.033,4.913,4.0,1.0
1,33.0,33.0,12.0,33.0,2.0,0.0,0.0,0.0,0.0,9.45,...,0.0,0.0,0.0,0.0,0.0,2.246,3.240,3.497,1.0,1.0
2,23.0,30.0,9.0,30.0,1.0,2.0,0.0,0.0,0.0,6.30,...,0.0,0.0,0.0,1.0,0.0,1.841,3.240,3.401,3.0,0.0
3,38.0,35.0,5.0,76.0,2.0,10.0,1.0,1.0,1.0,6.05,...,1.0,1.0,1.0,1.0,1.0,1.800,3.807,4.331,4.0,0.0
4,7.0,35.0,14.0,80.0,2.0,15.0,0.0,1.0,0.0,7.10,...,0.0,0.0,1.0,1.0,0.0,1.960,3.091,4.382,3.0,0.0
5,68.0,52.0,17.0,120.0,1.0,24.0,0.0,1.0,0.0,20.70,...,0.0,0.0,0.0,0.0,0.0,3.030,3.240,4.787,1.0,0.0
6,42.0,40.0,7.0,37.0,2.0,8.0,1.0,1.0,1.0,8.25,...,0.0,1.0,1.0,1.0,1.0,2.110,3.157,3.611,4.0,0.0


In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 28 columns):
tenure      200 non-null float64
age         200 non-null float64
address     200 non-null float64
income      200 non-null float64
ed          200 non-null float64
employ      200 non-null float64
equip       200 non-null float64
callcard    200 non-null float64
wireless    200 non-null float64
longmon     200 non-null float64
tollmon     200 non-null float64
equipmon    200 non-null float64
cardmon     200 non-null float64
wiremon     200 non-null float64
longten     200 non-null float64
tollten     200 non-null float64
cardten     200 non-null float64
voice       200 non-null float64
pager       200 non-null float64
internet    200 non-null float64
callwait    200 non-null float64
confer      200 non-null float64
ebill       200 non-null float64
loglong     200 non-null float64
logtoll     200 non-null float64
lninc       200 non-null float64
custcat     200 non-null float64
chur

In [127]:
df.describe()

,tenure,age,address,income,ed,employ,equip,callcard,wireless,longmon,...,pager,internet,callwait,confer,ebill,loglong,logtoll,lninc,custcat,churn
count,200.000000,200.000000,200.000000,200.000000,200.00000,200.00000,200.000000,200.000000,200.000000,200.00000,...,200.000000,200.000000,200.00000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,35.505000,41.165000,11.650000,75.130000,2.82500,10.22500,0.425000,0.705000,0.290000,11.78925,...,0.275000,0.440000,0.45500,0.460000,0.440000,2.193285,3.229185,3.951015,2.475000,0.290000
std,21.640971,13.076803,10.158419,128.430468,1.28555,8.95743,0.495584,0.457187,0.454901,9.88725,...,0.447635,0.497633,0.49922,0.499648,0.497633,0.731282,0.281019,0.752553,1.079445,0.454901
min,1.000000,19.000000,0.000000,9.000000,1.00000,0.00000,0.000000,0.000000,0.000000,1.10000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.095000,1.749000,2.197000,1.000000,0.000000
25%,16.750000,31.000000,3.000000,31.000000,2.00000,3.00000,0.000000,0.000000,0.000000,5.53750,...,0.000000,0.000000,0.00000,0.000000,0.000000,1.711750,3.226500,3.434000,2.000000,0.000000
50%,33.500000,40.000000,9.000000,48.000000,3.00000,7.50000,0.000000,1.000000,0.000000,8.25000,...,0.000000,0.000000,0.00000,0.000000,0.000000,2.110000,3.240000,3.871000,2.000000,0.000000
75%,55.250000,51.000000,18.000000,80.000000,4.00000,17.00000,1.000000,1.000000,1.000000,14.30000,...,1.000000,1.000000,1.00000,1.000000,1.000000,2.660000,3.240000,4.382000,3.000000,1.000000
max,72.000000,76.000000,48.000000,1668.000000,5.00000,44.00000,1.000000,1.000000,1.000000,62.30000,...,1.000000,1.000000,1.00000,1.000000,1.000000,4.132000,4.227000,7.419000,4.000000,1.000000


In [128]:
df.dtypes

tenure      float64
age         float64
address     float64
income      float64
ed          float64
employ      float64
equip       float64
callcard    float64
wireless    float64
longmon     float64
tollmon     float64
equipmon    float64
cardmon     float64
wiremon     float64
longten     float64
tollten     float64
cardten     float64
voice       float64
pager       float64
internet    float64
callwait    float64
confer      float64
ebill       float64
loglong     float64
logtoll     float64
lninc       float64
custcat     float64
churn       float64
dtype: object

# data prerocessing and selection

In [129]:
df.columns

Index(['tenure', 'age', 'address', 'income', 'ed', 'employ', 'equip',
       'callcard', 'wireless', 'longmon', 'tollmon', 'equipmon', 'cardmon',
       'wiremon', 'longten', 'tollten', 'cardten', 'voice', 'pager',
       'internet', 'callwait', 'confer', 'ebill', 'loglong', 'logtoll',
       'lninc', 'custcat', 'churn'],
      dtype='object')

In [130]:
features=x=df[['tenure', 'age', 'address', 'income', 'ed', 'employ', 'equip',
       'callcard', 'wireless', 'longmon', 'tollmon', 'equipmon', 'cardmon',
       'wiremon', 'longten', 'tollten', 'cardten', 'voice', 'pager',
       'internet', 'callwait', 'confer', 'ebill', 'loglong', 'logtoll',
       'lninc', 'custcat']].astype('int64')

x[0:7]


,tenure,age,address,income,ed,employ,equip,callcard,wireless,longmon,...,voice,pager,internet,callwait,confer,ebill,loglong,logtoll,lninc,custcat
0,11,33,7,136,5,5,0,1,1,4,...,1,1,0,1,1,0,1,3,4,4
1,33,33,12,33,2,0,0,0,0,9,...,0,0,0,0,0,0,2,3,3,1
2,23,30,9,30,1,2,0,0,0,6,...,0,0,0,0,1,0,1,3,3,3
3,38,35,5,76,2,10,1,1,1,6,...,1,1,1,1,1,1,1,3,4,4
4,7,35,14,80,2,15,0,1,0,7,...,1,0,0,1,1,0,1,3,4,3
5,68,52,17,120,1,24,0,1,0,20,...,0,0,0,0,0,0,3,3,4,1
6,42,40,7,37,2,8,1,1,1,8,...,1,0,1,1,1,1,2,3,3,4


In [131]:
target=y=df['churn'].astype('int64')

y.head(7)

0    1
1    1
2    0
3    0
4    0
5    0
6    0
Name: churn, dtype: int64

In [132]:
x=np.asanyarray(x)
x[0:7]

array([[  11,   33,    7,  136,    5,    5,    0,    1,    1,    4,   20,
           0,   15,   35,   42,  211,  125,    1,    1,    0,    1,    1,
           0,    1,    3,    4,    4],
       [  33,   33,   12,   33,    2,    0,    0,    0,    0,    9,    0,
           0,    0,    0,  288,    0,    0,    0,    0,    0,    0,    0,
           0,    2,    3,    3,    1],
       [  23,   30,    9,   30,    1,    2,    0,    0,    0,    6,    0,
           0,    0,    0,  157,    0,    0,    0,    0,    0,    0,    1,
           0,    1,    3,    3,    3],
       [  38,   35,    5,   76,    2,   10,    1,    1,    1,    6,   45,
          50,   23,   64,  239, 1873,  880,    1,    1,    1,    1,    1,
           1,    1,    3,    4,    4],
       [   7,   35,   14,   80,    2,   15,    0,    1,    0,    7,   22,
           0,   23,    0,   47,  166,  145,    1,    0,    0,    1,    1,
           0,    1,    3,    4,    3],
       [  68,   52,   17,  120,    1,   24,    0,    1,    0,   2

In [133]:
y=np.asanyarray(y)
y[0:7]

array([1, 1, 0, 0, 0, 0, 0], dtype=int64)

# tts

In [134]:
from sklearn.model_selection import train_test_split as tts


In [135]:
x_train,x_test,y_train,y_test=tts(x,y,test_size=0.25,random_state=4)

In [136]:
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(150, 27) (50, 27) (150,) (50,)


# logistic regression using newton-cg solver

In [156]:
from sklearn.linear_model import LogisticRegression as LR



In [172]:
lr=LR(C=0.02, solver='newton-cg')
lr                                # DEFAULT solver is lbfgs, we will go for newton-cg


LogisticRegression(C=0.02, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [173]:
lr.fit(x_train,y_train)

LogisticRegression(C=0.02, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [174]:
y_pred=lr.predict(x_test)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 1], dtype=int64)

In [175]:
from sklearn.metrics import jaccard_similarity_score as jsc

In [176]:
jsc(y_test,y_pred)

C:\Users\91755\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:635: DeprecationWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  'and multiclass classification tasks.', DeprecationWarning)


0.68

# logistic regression using the default lbfgs solver, using the same value of 
# C(INVERSE REGULARIZATION PARAMETER)

In [179]:
lr1=LR(C=0.02, solver='lbfgs')
lr1                               


LogisticRegression(C=0.02, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [184]:
lr1.fit(x_train,y_train)

C:\Users\91755\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=0.02, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [185]:
y_pred=lr1.predict(x_test)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 1], dtype=int64)

In [186]:
from sklearn.metrics import jaccard_similarity_score as jsc

In [187]:
jsc(y_test,y_pred)

C:\Users\91755\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:635: DeprecationWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  'and multiclass classification tasks.', DeprecationWarning)


0.7

# using liblinear solver, with inverse regularization parameter(C) as 0.02

In [188]:
lr2=LR(C=0.02, solver='liblinear')
lr2                               


LogisticRegression(C=0.02, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [189]:
lr2.fit(x_train,y_train)

LogisticRegression(C=0.02, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [190]:
y_pred=lr2.predict(x_test)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 1], dtype=int64)

In [191]:
from sklearn.metrics import jaccard_similarity_score as jsc

In [192]:
jsc(y_test,y_pred)

C:\Users\91755\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:635: DeprecationWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  'and multiclass classification tasks.', DeprecationWarning)


0.7

array([[9.57247541e-01, 4.27524593e-02],
       [9.95622618e-01, 4.37738192e-03],
       [7.87079653e-01, 2.12920347e-01],
       [9.42180422e-01, 5.78195779e-02],
       [7.73742774e-01, 2.26257226e-01],
       [5.66826502e-01, 4.33173498e-01],
       [7.70236924e-01, 2.29763076e-01],
       [7.42492680e-01, 2.57507320e-01],
       [4.20059347e-01, 5.79940653e-01],
       [9.98120990e-01, 1.87900980e-03],
       [6.54837640e-01, 3.45162360e-01],
       [9.91691655e-01, 8.30834547e-03],
       [7.37331395e-01, 2.62668605e-01],
       [4.90547590e-01, 5.09452410e-01],
       [9.50618405e-01, 4.93815953e-02],
       [8.05185425e-01, 1.94814575e-01],
       [5.29155620e-01, 4.70844380e-01],
       [9.99607262e-01, 3.92738003e-04],
       [7.27519669e-01, 2.72480331e-01],
       [9.47942832e-01, 5.20571678e-02],
       [5.55010747e-01, 4.44989253e-01],
       [5.21114585e-01, 4.78885415e-01],
       [9.99504443e-01, 4.95556759e-04],
       [1.27501915e-01, 8.72498085e-01],
       [5.488816

# from here we conclude that liblinear and lbfgs solvers are better for this data

In [203]:
y_prob = lr2.predict_proba(x_test)
y_prob[0:5]

array([[0.95708237, 0.04291763],
       [0.99553153, 0.00446847],
       [0.78054486, 0.21945514],
       [0.94262053, 0.05737947],
       [0.76340731, 0.23659269]])

In [204]:
from sklearn.metrics import log_loss
log_loss(y_test, y_prob)   #LOG LOSS VALUE


0.9007518341892716